<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 The MediaPipe Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train a custom object detection model with MediaPipe Model Maker

In this colab notebook, you'll learn how to use [MediaPipe Model Maker](https://developers.google.com/mediapipe/solutions/model_maker) to train a custom object detection model to detect dogs.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset to use with the [MediaPipe Object Detector task](https://developers.google.com/mediapipe/solutions/vision/object_detector).


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/google/mediapipe/tree/master/mediapipe/model_maker) and the pycocotools library you'll use for evaluation.

In [1]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow-metadata to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━

Import the required packages.

In [2]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from google.colab import files

from mediapipe_model_maker import object_detector

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   freedomtech.zip   images   images.zip  'My Drive'   pass


In [4]:
!unzip /mydrive/freedomtech.zip


Archive:  /mydrive/freedomtech.zip
   creating: freedomtech/train/
   creating: freedomtech/train/images/
  inflating: freedomtech/train/images/helmet_30.jpg  
  inflating: freedomtech/train/images/helmet_0.jpg  
  inflating: freedomtech/train/images/helmet_7.jpg  
  inflating: freedomtech/train/images/helmet_9.jpg  
  inflating: freedomtech/train/images/no_helmet_48.jpg  
  inflating: freedomtech/train/images/no_helmet_12.jpg  
  inflating: freedomtech/train/images/helmet_13.jpg  
  inflating: freedomtech/train/images/no_helmet_13.jpg  
  inflating: freedomtech/train/images/helmet_28.jpg  
  inflating: freedomtech/train/images/helmet_10.jpg  
  inflating: freedomtech/train/images/helmet_33.jpg  
  inflating: freedomtech/train/images/no_helmet_10.jpg  
  inflating: freedomtech/train/images/helmet_35.jpg  
  inflating: freedomtech/train/images/helmet_3.jpg  
  inflating: freedomtech/train/images/no_helmet_45.jpg  
  inflating: freedomtech/train/images/no_helmet_5.jpg  
  inflating: free

In [5]:
train_dataset_path = "/content/freedomtech/train"
validation_dataset_path = "/content/freedomtech/validate"

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [6]:
train_data = object_detector.Dataset.from_pascal_voc_folder(
    '/content/freedomtech/validate',
    cache_dir="/tmp/od_data/train",
)

val_data = object_detector.Dataset.from_pascal_voc_folder(
    '/content/freedomtech/validate',
    cache_dir="/tmp/od_data/validatation")

### Step 2: Train the TensorFlow model with the training data.

* Set `batch_size = 8` here so you will see that it takes 13 steps to go through the 139 images in the training dataset.
* Set `learning_rate = 0.3` here so the model will adjust its weights at a rate of 0.3.
* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.


In [7]:
hparams = object_detector.HParams(batch_size=4, learning_rate=0.3, epochs=100, export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=object_detector.SupportedModels.MOBILENET_V2,
    hparams=hparams
)
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=val_data,
    options=options)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


Model: "retina_net_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobile_net (MobileNet)      {'2': (None, 64, 64, 24   2257984   
                             ),                                  
                              '3': (None, 32, 32, 32             
                             ),                                  
                              '4': (None, 16, 16, 96             
                             ),                                  
                              '5': (None, 8, 8, 320)             
                             , '6': (None, 8, 8, 128             
                             0)}                                 
                                                                 
 fpn (FPN)                   {'5': (None, 8, 8, 128)   149056    
                             , '4': (None, 16, 16, 1             
                             28),                 

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


25/25 [==============================] - 62s 482ms/step - total_loss: 23.0230 - cls_loss: 22.6845 - box_loss: 0.0057 - model_loss: 22.9681 - val_total_loss: 1.5998 - val_cls_loss: 1.2592 - val_box_loss: 0.0057 - val_model_loss: 1.5448
Epoch 2/100
25/25 [==============================] - 5s 184ms/step - total_loss: 1.4488 - cls_loss: 1.1964 - box_loss: 0.0039 - model_loss: 1.3938 - val_total_loss: 1.4417 - val_cls_loss: 1.1898 - val_box_loss: 0.0039 - val_model_loss: 1.3867
Epoch 3/100
25/25 [==============================] - 5s 192ms/step - total_loss: 1.2535 - cls_loss: 1.0809 - box_loss: 0.0024 - model_loss: 1.1986 - val_total_loss: 1.2890 - val_cls_loss: 1.0776 - val_box_loss: 0.0031 - val_model_loss: 1.2341
Epoch 4/100
25/25 [==============================] - 4s 175ms/step - total_loss: 1.0325 - cls_loss: 0.9014 - box_loss: 0.0015 - model_loss: 0.9775 - val_total_loss: 1.0731 - val_cls_loss: 0.8941 - val_box_loss: 0.0025 - val_model_loss: 1.0181
Epoch 5/100
25/25 [=================

### Step 3. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 27 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [8]:
loss, coco_metrics = model.evaluate(val_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

25/25 [==============================] - 1s 34ms/step - total_loss: 0.3323 - cls_loss: 0.2148 - box_loss: 0.0013 - model_loss: 0.2775
creating index...
index created!
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.34s).
Accumulating evaluation results...
DONE (t=0.09s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.693
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.955
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.892
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.161
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.707
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.787
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.739
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.740
 Average Recall     (AR) @[ I

### Step 4: Export as a TensorFlow Lite model.

After creating the model, convert and export it to a Tensorflow Lite model format, and then download it for later use in an on-device application with MediaPipe Tasks.

In [9]:
model.export_model('best.tflite')
!ls exported_model
files.download('exported_model/best.tflite')

Exporting a floating point model


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['6'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


best.tflite  float_ckpt.data-00000-of-00001  float_ckpt.index  metadata.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>